# Goals of this script:
1. Convert TCX file to TXT file with position data
2. Write out a summary of workout info to compare with what was downloaded
---
## Next steps after this works:
3. Identify which TCX files are mislabled => Relabel or re-download them
4. Add field to database to indicate which TCX files are correctly converted to TXT
5. Finish downloading the rest of existing TCX files and convert
6. Quick research about how to store spatial data
7. Can I visualize the converted data?
---
## Prereqs: 
- Need to have TCX files downloaded locally  (see Automator script)
- To do that, I used the workoutIds found in the CSV file of exported workout_history from MapMyRun

## Preprocessing & Helper functions: 
- Setup a list of TCX files to loop through & label summary file
- Define Helper funcitons

In [1]:
import os

mainfolder = '/Users/barbaraihidalgo-sotelo/PROJECTS/build_workout_dashboard/build_workout_dashboard/'
summaryfile = mainfolder + 'workout_position_summary.csv'

# Label summary file, if it hasnt been created yet
if not(os.path.isfile(summaryfile)):
    fh = open(summaryfile,"a+")
    fh.writelines('WorkoutId,TimeStamp,Calories,TotalTimeSeconds,MaximumSpeed,#TxtFileRows,#SamplingInterval'+'\n')
    fh.close()

In [2]:
import datetime as dt
from scipy import stats

def calc_Time(Time):
    timestamps, dates, times = [], [], []
    for samp in Time:
        date_time_obj = dt.datetime.strptime(samp[0:19].replace('T',' '), '%Y-%m-%d %H:%M:%S') 
        timestamps.append(date_time_obj.date())
        dates.append(date_time_obj.time())
        times.append(date_time_obj)

    # Calculate sampling interval
    sampInt = []
    for i in range(1,len(times)):
        sampInt.append((times[i] - times[i-1]).total_seconds())
    typSampInterval = int(stats.mode(sampInt)[0])

    return typSampInterval
#     return typSampInterval, timestamps, dates, times

#     print('Date:', date_time_obj.date())
#     print('Time:', date_time_obj.time())
#     print('Date-time:', date_time_obj)


In [3]:
def extractSummaryInfo(root):
    base = '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}'

    TimeStamp = [m.text for m in root.iter(base+'Id')]
    Calories = [m.text for m in root.iter(base+'Calories')]
    TotalTimeSec = [m.text for m in root.iter(base+'TotalTimeSeconds')]
    MaxSpeed = [m.text for m in root.iter(base+'MaximumSpeed')]
    if len(TimeStamp)==0:
        TimeStamp = ''
    else:
        TimeStamp = TimeStamp[0]
        # Trip Timestamp to something easier to convert to datetime format
        TimeStamp = TimeStamp[0:19].replace('T',' ')
    if len(Calories)==0:
        Calories = ''
    else:
        Calories = Calories[0]    
    if len(TotalTimeSec)==0:
        TotalTimeSec = ''
    else:
        TotalTimeSec = TotalTimeSec[0] 
    if len(MaxSpeed)==0:
        MaxSpeed = ''
    else:
        MaxSpeed = MaxSpeed[0]  
    
    return TimeStamp, Calories, TotalTimeSec, MaxSpeed


In [4]:
def extractPositionData(root):
    base = '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}'
    Time, Pos, LatPts, LongPts, AltM, DistM = [], [], [], [], [], []
    [Time.append(m.text) for m in root.iter(base+'Time')]
    [Pos.append(m.text) for m in root.iter(base+'Position')]
    [LatPts.append(m.text) for m in root.iter(base+'LatitudeDegrees')]
    [LongPts.append(m.text) for m in root.iter(base+'LongitudeDegrees')]
    [AltM.append(m.text) for m in root.iter(base+'AltitudeMeters')]
    [DistM.append(m.text) for m in root.iter(base+'DistanceMeters')]

    if len(DistM)<1:
        return '', '', '', '', '', ''
    
    elif float(DistM[0]) > 10:
#         # Check whether first element in the list is the TOTAL distance meters
#         print(f'(first 3 elements: {DistM[0]}, {DistM[1]}, {DistM[2]})')
        TotalDistMeters = DistM[0]
        DistM = DistM[1:]
    
    TimeConv = []
    for samp in Time:
        TimeConv.append(samp[0:19].replace('T',' '))
    
    return TimeConv, Pos, LatPts, LongPts, AltM, DistM

In [5]:
def qualityChecks(Time, Pos, LatPts, LongPts, AltM, DistM):
    print('Time: ' + str(len(Time)))
    print('Position: ' + str(len(Pos)))
    print('LatDeg: ' + str(len(LatPts)))
    print('LongDeg: ' + str(len(LongPts)))
    print('AltitudeMeters: ' + str(len(AltM)))
    print('DistanceMeters: ' + str(len(DistM)))

#     print(f"\nNote: Length of these lists should be roughly equal to TotalTimeSeconds, {TotalTimeSec}")
    # print(f"Total Time (seconds): {TotalTimeSec}")
#     print('(But not always old files greater than 1sec btwn samples)')

    return Time, Pos, LatPts, LongPts, AltM, DistM

In [6]:
def writePositionFile(writefile, Time, LatPts, LongPts, AltM, DistM):
    # Save position data
    numLines=0
    for l in range(0,len(Time)):
        # Open file handle to write data:
        fh = open(writefile,"a+")
        # Write data
        try:
            L = Time[l]+'\t'+LatPts[l]+'\t'+LongPts[l]+'\t'+AltM[l]+'\t'+DistM[l]+'\n'
        except IndexError:
        # print('There was an index error; probably one array is shorter than others')
            if len(LatPts)<=numLines:
                LatTemp = 'NaN'
            else:
                LatTemp = LatPts[l]
            if len(LongPts)<=numLines:
                LonTemp = 'NaN'
            else:
                LonTemp = LongPts[l]
            if len(AltM)<=numLines:
                AltMTemp = 'NaN'
            else:
                AltMTemp = AltM[l]
            if len(DistM)<=numLines:
                DistMTemp = 'NaN'
            else:
                DistMTemp =  DistM[l]
            L = Time[l]+'\t'+LatTemp+'\t'+LonTemp+'\t'+AltMTemp+'\t'+DistMTemp+'\n'
        fh.writelines(L)
        # Close file handle
        fh.close()
        numLines+=1
    return numLines
    
def writeSummaryData(summaryfile,workoutId,TimeStamp,Calories,TotalTimeSec,MaxSpeed,numLines,sampInterval):
    # Save summary data
    fh = open(summaryfile,"a+")

    # Write data
    L = workoutId+','+TimeStamp+','+Calories+','+TotalTimeSec+','+MaxSpeed+','+str(numLines)+' lines in txt file'+','+str(sampInterval)+'\n'
    fh.writelines(L)

    # Close file handle
    fh.close()


## Loop through TCX files...
1: Load data  
2: Extract summary info  
3: Extract position info  
4: Quality data checks  
5: Write out text file   
- Write a new file for workout position data ("workoutId.txt")
- Append summary stats to file with summary workout info (workout_position_summary.csv)

__NOTE__:  
If you run the cell below again, it'll add a new row to the workout_summary for each of the workouts that doesn't have a .txt file (i.e. no position data) ...  
- As of Jan 2021, there were 39 of those, all except two from 2011 and 2013
- That leaves 969 workouts with position data (as of 1/1/2021)

In [9]:
import lxml.etree as ET

readfolder = "/Users/barbaraihidalgo-sotelo/PROJECTS/build_workout_dashboard/workouts_timedata/"
writefolder = "/Users/barbaraihidalgo-sotelo/PROJECTS/build_workout_dashboard/workouts_timedata_TXT/"

# Identify number of TCX files in folder
nFiles=0
for file in os.listdir(readfolder): 
    if file.endswith('.tcx'):
        nFiles+=1 

# Check each of them in this loop
cnt=0
for file in os.listdir(readfolder): 
    if file.endswith('.tcx'):
        cnt+=1
        print(f"Processing file {cnt} of {nFiles}")
        workoutId = file[0:-4]

        # Check whether this data exists already
        writefile = writefolder + workoutId + '.txt'
        if not(os.path.isfile(writefile)):

            # Label new file
            fh = open(writefile,"a+")
            fh.writelines('TimeStamp\tLatitude\tLongitude\tAltitude(m)\tDistance(m)\n')
            fh.close()

            # LOAD XML FILE:
            tree =  ET.parse(readfolder + file)

            # Identify top-level element:
            root = tree.getroot();

            # --- TO GET FAMILIAR WITH FILE CONTENTS: --- 
            # Print XML in "readable" format
            # print(ET.tostring(root, pretty_print=True).strip())  

            # These are the elements in the XML tree:
            # xmlElem = [elem.tag for elem in root.iter()]
            # print(set(xmlElem))
            
            # Extract workout summary info:
            TimeStamp,Calories,TotalTimeSec,MaxSpeed = extractSummaryInfo(root)
            print('---------------------------------')
            print(f"Workout id: {workoutId} \nDate/Time: {TimeStamp} \nCalories: {Calories} \nTotal Time (seconds): {TotalTimeSec} \nMax Speed (seconds): {MaxSpeed}")

            # Extract position data:
            Time, Pos, LatPts, LongPts, AltM, DistM = extractPositionData(root)
            # Check whether any position data exists:
            if len(Time) == 0:
                os.remove(writefile)
                print(f'No data to write to {writefile}')
                numLines, sampInterval = 'NaN','NaN'
            else:
                # (wonky workaround for when there's one extra Time row)
                if (len(Time)-len(Pos)) == 1:
                    Time = Time[0:-1]  

                # Quality checks:
                #qualityChecks(Time, Pos, LatPts, LongPts, AltM, DistM)
                                
                # Write Position Data 
                numLines = writePositionFile(writefile, Time, LatPts, LongPts, AltM, DistM)
                print(f"Saved txt file with {numLines} lines")

                # Add sample interval
                sampInterval = calc_Time(Time)

            # Write Summary Info 
            writeSummaryData(summaryfile,workoutId,TimeStamp,Calories,TotalTimeSec,MaxSpeed,numLines, sampInterval)
            #print(f"Appended workout {workoutId}, {TotalTimeSec} sec, to summary data\n")


Processing file 1 of 303
Processing file 2 of 303
---------------------------------
Workout id: 2159032526 
Date/Time: 2017-04-29 18:27:59 
Calories: 307.999999822592 
Total Time (seconds): 1223.0 
Max Speed (seconds): 3.7427261695999996
Saved txt file with 235 lines
Processing file 3 of 303
---------------------------------
Workout id: 748644425 
Date/Time: 2014-09-28 21:12:05 
Calories: 212.999999877312 
Total Time (seconds): 1001.0 
Max Speed (seconds): 
Saved txt file with 248 lines
Processing file 4 of 303
---------------------------------
Workout id: 3323018935 
Date/Time: 2019-01-09 04:06:37 
Calories: 137.99999992051198 
Total Time (seconds): 1697.0 
Max Speed (seconds): 2.6296859776
Saved txt file with 3698 lines
Processing file 5 of 303
---------------------------------
Workout id: 4293094438 
Date/Time: 2020-05-16 02:21:37 
Calories: 57.999999966592 
Total Time (seconds): 1139.0 
Max Speed (seconds): 2.1076192544000003
Saved txt file with 1113 lines
Processing file 6 of 303


Saved txt file with 2258 lines
Processing file 159 of 1284
---------------------------------
Workout id: 976773295 
Date/Time: 2015-05-08 17:54:12 
Calories: 752.999999566272 
Total Time (seconds): 3199.0 
Max Speed (seconds): 4.532493855999999
Saved txt file with 839 lines
Processing file 160 of 1284
---------------------------------
Workout id: 3437769625 
Date/Time: 2019-03-18 00:16:26 
Calories: 286.99999983468797 
Total Time (seconds): 3838.0 
Max Speed (seconds): 3.4565490432
Saved txt file with 7696 lines
Processing file 161 of 1284
---------------------------------
Workout id: 4019562331 
Date/Time: 2020-02-07 03:44:12 
Calories: 173.999999899776 
Total Time (seconds): 2395.0 
Max Speed (seconds): 2.998118464
Saved txt file with 2375 lines
Processing file 162 of 1284
Processing file 163 of 1284
---------------------------------
Workout id: 888195407 
Date/Time: 2015-02-28 01:32:04 
Calories: 869.99999949888 
Total Time (seconds): 3600.0 
Max Speed (seconds): 
No data to write t

Saved txt file with 15251 lines
Processing file 198 of 1284
---------------------------------
Workout id: 3289342183 
Date/Time: 2018-12-15 23:49:22 
Calories: 286.99999983468797 
Total Time (seconds): 4162.0 
Max Speed (seconds): 2.4750861343999997
Saved txt file with 8589 lines
Processing file 199 of 1284
---------------------------------
Workout id: 4636849009 
Date/Time: 2020-08-31 04:21:15 
Calories: 202.999999883072 
Total Time (seconds): 2962.0 
Max Speed (seconds): 12.582656064
Saved txt file with 13227 lines
Processing file 200 of 1284
Processing file 201 of 1284
---------------------------------
Workout id: 1717697867 
Date/Time: 2016-09-01 01:09:46 
Calories: 831.999999520768 
Total Time (seconds): 3735.0 
Max Speed (seconds): 7.08603104
Saved txt file with 977 lines
Processing file 202 of 1284
---------------------------------
Workout id: 4110363271 
Date/Time: 2020-03-27 19:56:13 
Calories: 638.999999631936 
Total Time (seconds): 3336.0 
Max Speed (seconds): 3.902878249599

Saved txt file with 5002 lines
Processing file 241 of 1284
---------------------------------
Workout id: 4075292791 
Date/Time: 2020-03-11 17:13:55 
Calories: 145.999999915904 
Total Time (seconds): 2963.0 
Max Speed (seconds): 2.3512337023999996
Saved txt file with 2959 lines
Processing file 242 of 1284
---------------------------------
Workout id: 4554423265 
Date/Time: 2020-08-03 01:33:06 
Calories: 122.999999929152 
Total Time (seconds): 2196.0 
Max Speed (seconds): 1.7500096064
Saved txt file with 9871 lines
Processing file 243 of 1284
---------------------------------
Workout id: 2255794769 
Date/Time: 2017-06-11 19:34:07 
Calories: 558.999999678016 
Total Time (seconds): 2669.0 
Max Speed (seconds): 4.38691528
Saved txt file with 646 lines
Processing file 244 of 1284
---------------------------------
Workout id: 1043527607 
Date/Time: 2015-06-24 16:19:42 
Calories: 409.99999976384 
Total Time (seconds): 1830.0 
Max Speed (seconds): 5.157455776
Saved txt file with 477 lines
Proce

Saved txt file with 2919 lines
Processing file 277 of 1284
---------------------------------
Workout id: 1368350789 
Date/Time: 2016-03-02 23:51:51 
Calories: 364.99999978976 
Total Time (seconds): 1652.0 
Max Speed (seconds): 4.7801093119999996
Saved txt file with 425 lines
Processing file 278 of 1284
---------------------------------
Workout id: 710720781 
Date/Time: 2014-08-29 20:13:45 
Calories: 370.999999786304 
Total Time (seconds): 1677.0 
Max Speed (seconds): 
Saved txt file with 1158 lines
Processing file 279 of 1284
---------------------------------
Workout id: 3044920561 
Date/Time: 2018-07-27 23:34:26 
Calories: 498.99999971257597 
Total Time (seconds): 2325.0 
Max Speed (seconds): 7.503968736000001
Saved txt file with 3477 lines
Processing file 280 of 1284
---------------------------------
Workout id: 1665376166 
Date/Time: 2016-08-09 05:03:42 
Calories: 180.999999895744 
Total Time (seconds): 2428.0 
Max Speed (seconds): 2.9472140192
Saved txt file with 342 lines
Processi

...

Saved txt file with 3494 lines
Processing file 1100 of 1284
Processing file 1101 of 1284
---------------------------------
Workout id: 5026085251 
Date/Time: 2020-11-25 22:36:14 
Calories: 859.99999950464 
Total Time (seconds): 4592.0 
Max Speed (seconds): 5.540703167999999
Saved txt file with 25266 lines
Processing file 1102 of 1284
Processing file 1103 of 1284
---------------------------------
Workout id: 4290652828 
Date/Time: 2020-05-15 02:55:00 
Calories: 166.999999903808 
Total Time (seconds): 3010.0 
Max Speed (seconds): 4.771973184
Saved txt file with 2977 lines
Processing file 1104 of 1284
---------------------------------
Workout id: 4151842324 
Date/Time: 2020-04-09 15:56:01 
Calories: 164.99999990496 
Total Time (seconds): 3001.0 
Max Speed (seconds): 2.0587667232
Saved txt file with 3002 lines
Processing file 1105 of 1284
---------------------------------
Workout id: 2563521571 
Date/Time: 2017-11-07 00:05:05 
Calories: 558.999999678016 
Total Time (seconds): 2673.0 
Max S

Saved txt file with 1779 lines
Processing file 1142 of 1284
---------------------------------
Workout id: 2768776088 
Date/Time: 2018-03-17 20:20:26 
Calories: 680.999999607744 
Total Time (seconds): 3107.0 
Max Speed (seconds): 6.818388192
Saved txt file with 4632 lines
Processing file 1143 of 1284
---------------------------------
Workout id: 4994048491 
Date/Time: 2020-11-21 00:42:28 
Calories: 226.999999869248 
Total Time (seconds): 3162.0 
Max Speed (seconds): 2.7368101728
Saved txt file with 14777 lines
Processing file 1144 of 1284
---------------------------------
Workout id: 649182613 
Date/Time: 2014-07-16 16:26:36 
Calories: 371.999999785728 
Total Time (seconds): 2461.0 
Max Speed (seconds): 
Saved txt file with 1335 lines
Processing file 1145 of 1284
---------------------------------
Workout id: 4063702276 
Date/Time: 2020-03-05 01:52:31 
Calories: 287.999999834112 
Total Time (seconds): 4032.0 
Max Speed (seconds): 2.6230876672
Saved txt file with 4031 lines
Processing fil

Saved txt file with 3234 lines
Processing file 1180 of 1284
---------------------------------
Workout id: 454334967 
Date/Time: 2014-01-01 18:42:00 
Calories: 922.9999994683519 
Total Time (seconds): 4010.0 
Max Speed (seconds): 
Saved txt file with 2861 lines
Processing file 1181 of 1284
---------------------------------
Workout id: 4789154608 
Date/Time: 2020-10-25 02:18:37 
Calories: 327.999999811072 
Total Time (seconds): 6563.0 
Max Speed (seconds): 2.5743916
Saved txt file with 26133 lines
Processing file 1182 of 1284
---------------------------------
Workout id: 4168800013 
Date/Time: 2020-04-14 17:25:42 
Calories: 221.999999872128 
Total Time (seconds): 2974.0 
Max Speed (seconds): 9.632683808
Saved txt file with 2968 lines
Processing file 1183 of 1284
---------------------------------
Workout id: 634783531 
Date/Time: 2014-07-01 16:17:24 
Calories: 443.999999744256 
Total Time (seconds): 1922.0 
Max Speed (seconds): 
Saved txt file with 1455 lines
Processing file 1184 of 1284


Processing file 1219 of 1284
Processing file 1220 of 1284
---------------------------------
Workout id: 873755361 
Date/Time: 2015-02-12 16:37:48 
Calories: 260.999999849664 
Total Time (seconds): 1237.0 
Max Speed (seconds): 4.0167840416
Saved txt file with 303 lines
Processing file 1221 of 1284
Processing file 1222 of 1284
---------------------------------
Workout id: 5095279492 
Date/Time: 2020-12-08 23:41:44 
Calories: 197.999999885952 
Total Time (seconds): 2956.0 
Max Speed (seconds): 3.6497507904
Saved txt file with 13962 lines
Processing file 1223 of 1284
---------------------------------
Workout id: 5082711880 
Date/Time: 2020-12-07 01:06:41 
Calories: 318.999999816256 
Total Time (seconds): 2156.0 
Max Speed (seconds): 
Saved txt file with 3282 lines
Processing file 1224 of 1284
Processing file 1225 of 1284
---------------------------------
Workout id: 4056279790 
Date/Time: 2020-02-29 03:49:17 
Calories: 250.999999855424 
Total Time (seconds): 3584.0 
Max Speed (seconds): 2.

Saved txt file with 2043 lines
Processing file 1265 of 1284
---------------------------------
Workout id: 875089241 
Date/Time: 2015-02-14 01:13:20 
Calories: 725.999999581824 
Total Time (seconds): 3105.0 
Max Speed (seconds): 4.454217152
Saved txt file with 810 lines
Processing file 1266 of 1284
Processing file 1267 of 1284
---------------------------------
Workout id: 3348592219 
Date/Time: 2019-01-25 04:43:14 
Calories: 110.999999936064 
Total Time (seconds): 1648.0 
Max Speed (seconds): 2.7901018112
Saved txt file with 3178 lines
Processing file 1268 of 1284
---------------------------------
Workout id: 3271935781 
Date/Time: 2018-12-01 21:50:41 
Calories: 729.99999957952 
Total Time (seconds): 3410.0 
Max Speed (seconds): 5.085124704
Saved txt file with 11007 lines
Processing file 1269 of 1284
Processing file 1270 of 1284
---------------------------------
Workout id: 4448664739 
Date/Time: 2020-06-27 02:03:14 
Calories: 189.99999989056 
Total Time (seconds): 2764.0 
Max Speed (se

### Notes

In [8]:
# workoutId,TimeStamp,Calories,TotalTimeSec,MaxSpeed,numLines
TotalTimeSec
TimeStamp

'2013-07-16 04:34:54'

In [9]:
# --- Get Position Info ---
# Approach 1 => use etree with "iter" to get tags directly
# # 
# Time, Pos, LatPts, LongPts, AltM, DistM = [], [], [], [], [], []

# [Time.append(m.text) for m in root.iter(base+'Time')]
# [Pos.append(m.text) for m in root.iter(base+'Position')]
# [LatPts.append(m.text) for m in root.iter(base+'LatitudeDegrees')]
# [LongPts.append(m.text) for m in root.iter(base+'LongitudeDegrees')]
# [AltM.append(m.text) for m in root.iter(base+'AltitudeMeters')]
# [DistM.append(m.text) for m in root.iter(base+'DistanceMeters')]

# # Check whether first element in the list is the TOTAL distance meters
# if float(DistM[0]) > 10:
#     print('The first element of DistanceMeters is probably the summary...')
#     print('Thus removing it from array')
#     print(f'(first 3 elements: {DistM[0]}, {DistM[1]}, {DistM[2]})')
#     TotalDistMeters = DistM[0]
#     DistM = DistM[1:]

# Potential Approach 2 => use etree with "iter" to get children of a higher level tag')
# for trackPt in root.iter(base + 'Trackpoint'):
#     for child in trackPt:
#         print(child.tag,'----', child.text)

# Potential Approach 3 => Use RegEx